# Import

In [38]:
import numpy as np
import pandas as pd
import polars as pl
import pickle
import torch
import os
import gc
import scipy.signal
import math
import json
import matplotlib.colors as mcolors
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import wandb
from types import SimpleNamespace
import _MultiResUNet as MultiResUNet
import torch.nn as nn
import torch.optim as optim

In [ ]:
SAMPLE_RATE = 200  # Hz
SAMPLE_RATE_TARGET = 50  # Hz

MAX_TIME = 50        # sec
MAX_LENGTH = SAMPLE_RATE * MAX_TIME  # length of the sequence

WINDOW_SIZES = [0.3, 0.6, 1.2]  # 초 단위 윈도우 크기

BATCH_SIZE = 4

In [41]:
# Hyperparameters for model
config = SimpleNamespace(
    SAVE_DIR='model',
    model_depth=8,
    model_width=32,
    kernel_size=5,
    problem_type='Classification',
    ds=True,
    ae=False,
    feature_number=512,
    is_transconv=True,

    learning_rate=0.00001,
)

In [43]:
# Hyperparameters for model
def save_config(config, save_path):
    with open(save_path, 'w') as f:
        json.dump(config.__dict__, f, indent=4)
        
save_config(config, os.path.join(config.SAVE_DIR, '2_config.json'))

In [ ]:
# MAX_LENGTH_TARGET를 2 ** model_depth의 배수로 설정
factor = 2 ** config.model_depth
MAX_LENGTH_TARGET = math.ceil((SAMPLE_RATE_TARGET * MAX_TIME) / factor) * factor
print(f'Max recording time: {MAX_LENGTH_TARGET/SAMPLE_RATE_TARGET} sec')
# MAX_LENGTH_TARGET = SAMPLE_RATE_TARGET * MAX_TIME  # length of the sequence

## 2 ** n 형태로 만들기
# raw_max_length_target = SAMPLE_RATE_TARGET * MAX_TIME
# MAX_LENGTH_TARGET = 2 ** math.ceil(math.log2(raw_max_length_target))

# Load Data

In [4]:
# 데이터 불러오기
X_data = np.load('X_data.npy', allow_pickle=True)
Y_data = np.load('Y_data.npy', allow_pickle=True)

print('X_data shape:', X_data.shape)
print('Y_data shape:', Y_data.shape)

X_data shape: (2990,)
Y_data shape: (2990,)


# Feature Engineering

In [53]:
class TimeSeriesFeatureEngineer:
    def __init__(self, window_sizes, sampling_rate):
        self.window_sizes = np.dot(window_sizes, sampling_rate).astype(int)
        self.encoder = None
        self.label_mapping = {
            'idle': 'walk',
            'rampascent': 'rampascent',
            'rampascent-walk': 'rampascent',
            'rampdescent': 'rampdescent',
            'rampdescent-walk': 'rampdescent',
            'stairascent': 'stairascent',
            'stairascent-walk': 'stairascent',
            'stairdescent': 'stairdescent',
            'stairdescent-walk': 'stairdescent',
            'stand': 'walk',
            'stand-walk': 'walk',
            'turn1': 'walk',
            'turn2': 'walk',
            'walk': 'walk',
            'walk-rampascent': 'rampascent',
            'walk-rampdescent': 'rampdescent',
            'walk-stairascent': 'stairascent',
            'walk-stairdescent': 'stairdescent',
            'walk-stand': 'walk'
        }

    def map_labels(self, Y_data):
        Y_data_mapped = []
        for y_seq in Y_data:
            Y_data_mapped.append(np.array([self.label_mapping[label] for label in y_seq]))
        return Y_data_mapped

    def create_encoder(self, Y_data):
        # 라벨 매핑
        Y_data_mapped = self.map_labels(Y_data)
        
        # 전체 라벨 수집
        all_labels = np.concatenate(Y_data_mapped)
        all_labels_unique = np.unique(all_labels).reshape(-1, 1)
        
        # OneHotEncoder를 사용하여 라벨 인코딩
        self.encoder = OneHotEncoder(sparse_output=False)
        self.encoder.fit(all_labels_unique)

        # 인코더의 라벨 출력
        print("Encoder classes:", self.encoder.categories_)
        return self.encoder

    def fit_transform_labels(self, Y_data):
        if self.encoder is None:
            raise ValueError("Encoder has not been created. Call create_encoder first.")
        
        # 라벨 매핑
        Y_data_mapped = self.map_labels(Y_data)
        
        # 각 Y_data를 원핫 인코딩
        Y_data_encoded_list = [self.encoder.transform(np.array(y).reshape(-1, 1)) for y in Y_data_mapped]
        return Y_data_encoded_list

    def feature_engineering(self, df: pl.DataFrame):
        # LazyFrame으로 변환하여 작업
        lf = df.lazy()
        
        for col in df.columns:
            for window in self.window_sizes:
                window_str = str(window)
                # 통계 값
                lf = lf.with_columns([
                    df[col].rolling_mean(window).alias(col + '_mean_' + window_str),
                    df[col].rolling_std(window).alias(col + '_std_' + window_str),
                    df[col].rolling_min(window).alias(col + '_min_' + window_str),
                    df[col].rolling_max(window).alias(col + '_max_' + window_str),
                    df[col].diff(window).alias(col + '_diff_' + window_str)
                ])
                for lag in range(1, 4):
                    lf = lf.with_columns([
                        df[col].shift(lag * window).alias(col + f'_lag_{lag}_' + window_str)
                    ])
        
        features_df = lf.collect().fill_nan(0).fill_null(0)
        return features_df

    def fit_transform_features(self, X_data):
        X_features = []
        for seq in X_data:
            seq_df = pl.DataFrame(seq)
            features_df = self.feature_engineering(seq_df)
            X_features.append(features_df.to_numpy())
        return X_features

    def resample_data(self, X_data, original_sampling_rate, target_sampling_rate):
        resampled_X_data = []
        for seq in X_data:
            resampled_seq = scipy.signal.resample(seq, int(len(seq) * target_sampling_rate / original_sampling_rate))
            resampled_X_data.append(resampled_seq)
        return resampled_X_data

    def fit(self, X_data, Y_data, original_sampling_rate, target_sampling_rate, train_dir="train_batches", val_dir="val_batches", test_size=0.2, max_workers=4):
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(val_dir, exist_ok=True)

        # Resample the data
        X_data_resampled = self.resample_data(X_data, original_sampling_rate, target_sampling_rate)

        # Statistics
        sequence_length = [len(seq) for seq in X_data_resampled]
        print(f'Max sequence length: {max(sequence_length)}')
        print(f'Min sequence length: {min(sequence_length)}')
        print(f'Mean sequence length: {np.mean(sequence_length)}')

        # Train/Val split
        X_train, X_val, Y_train, Y_val = train_test_split(X_data_resampled, Y_data, test_size=test_size, random_state=42)

        # 라벨 인코딩
        self.create_encoder(Y_data)
        Y_train_encoded = self.fit_transform_labels(Y_train)
        Y_val_encoded = self.fit_transform_labels(Y_val)

        # Train 데이터 저장
        self._process_and_save_individual(X_train, Y_train_encoded, train_dir, max_workers)
        # Val 데이터 저장
        self._process_and_save_individual(X_val, Y_val_encoded, val_dir, max_workers)


    def _process_and_save_individual(self, X_data, Y_data, save_dir, max_workers):
        def process_and_save(idx):
            X_features = self.fit_transform_features([X_data[idx]])[0]
            Y_encoded = Y_data[idx]
            
            with open(os.path.join(save_dir, f"X_data_{idx}.pkl"), 'wb') as f:
                pickle.dump(X_features, f)
            with open(os.path.join(save_dir, f"Y_data_{idx}.pkl"), 'wb') as f:
                pickle.dump(Y_encoded, f)
            
            del X_features, Y_encoded
            gc.collect()

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(process_and_save, idx) for idx in range(len(X_data))]
            for _ in tqdm(as_completed(futures), total=len(futures), desc=f"Processing data in {save_dir}", unit="sample"):
                pass

In [ ]:
feature_engineer = TimeSeriesFeatureEngineer(WINDOW_SIZES, SAMPLE_RATE_TARGET)

In [ ]:
feature_engineer.fit(X_data, Y_data, SAMPLE_RATE, SAMPLE_RATE_TARGET, max_workers=16)

# Dataloader

In [23]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X_dir, Y_dir, num_samples, max_length):
        self.X_dir = X_dir
        self.Y_dir = Y_dir
        self.num_samples = num_samples
        self.max_length = max_length

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        with open(os.path.join(self.X_dir, f"X_data_{idx}.pkl"), 'rb') as f:
            X_data = pickle.load(f)
        with open(os.path.join(self.Y_dir, f"Y_data_{idx}.pkl"), 'rb') as f:
            Y_data = pickle.load(f)

        X_padded = self.pad_or_trim_sequence(X_data)
        Y_padded = self.pad_or_trim_sequence(Y_data)
        
        return X_padded, Y_padded

    def pad_or_trim_sequence(self, sequence):
        seq_len = len(sequence)
        feature_dim = sequence.shape[1] if len(sequence.shape) > 1 else 1

        if seq_len > self.max_length:
            return torch.tensor(sequence[:self.max_length], dtype=torch.float32)
        else:
            padding_length = self.max_length - seq_len
            if feature_dim > 1:
                padded_seq = np.pad(sequence, ((0, padding_length), (0, 0)), 'constant', constant_values=0)
            else:
                padded_seq = np.pad(sequence, (0, padding_length), 'constant', constant_values=0)
            return torch.tensor(padded_seq, dtype=torch.float32)


In [24]:
# Create datasets
num_batches_train = len(os.listdir("train_batches")) // 2  # assuming one X and one Y file per batch
num_batches_val = len(os.listdir("val_batches")) // 2

train_dataset = TimeSeriesDataset(X_dir="train_batches", Y_dir="train_batches", num_samples=num_batches_train, max_length=MAX_LENGTH_TARGET)
val_dataset = TimeSeriesDataset(X_dir="val_batches", Y_dir="val_batches", num_samples=num_batches_val, max_length=MAX_LENGTH_TARGET)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [25]:
# for X_batch, Y_batch in train_loader:
#     print(X_batch.shape, Y_batch.shape)
#     pass

# Training

In [7]:
# 데이터 로더를 사용하여 모델의 길이, 채널 수 및 출력 채널 수 설정
first_batch = next(iter(train_loader))
length = first_batch[0].shape[1]
num_channel = first_batch[0].shape[2]
output_channels = first_batch[1].shape[-1]

In [8]:
# 모델, 손실 함수 및 옵티마이저 정의
model = MultiResUNet.UNet(length=length, model_depth=config.model_depth, num_channel=num_channel, model_width=config.model_width, kernel_size=config.kernel_size, problem_type=config.problem_type, output_channels=output_channels, ds=config.ds, ae=config.ae, feature_number=config.feature_number, is_transconv=config.is_transconv)

criterion = torch.nn.BCEWithLogitsLoss()  # 손실 함수 정의
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)  # 옵티마이저 정의

In [9]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, save_dir='model_checkpoints'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    scaler = GradScaler()

    best_val_loss = float('inf')
    os.makedirs(save_dir, exist_ok=True)

    pbar = tqdm(total=num_epochs, desc="Training model", unit="epoch")

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for X_batch, Y_batch in train_loader:
            X_batch = X_batch.to(device)
            Y_batch = Y_batch.to(device)

            optimizer.zero_grad()

            with autocast():
                outputs = model(X_batch)
                if isinstance(outputs, list):  # Deep Supervision
                    loss = sum([criterion(output, Y_batch) for output in outputs]) / len(outputs)
                else:
                    loss = criterion(outputs, Y_batch)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * X_batch.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, Y_batch in val_loader:
                X_batch = X_batch.to(device)
                Y_batch = Y_batch.to(device)

                with autocast():
                    outputs = model(X_batch)
                    if isinstance(outputs, list):  # Deep Supervision
                        loss = sum([criterion(output, Y_batch) for output in outputs]) / len(outputs)
                    else:
                        loss = criterion(outputs, Y_batch)

                val_loss += loss.item() * X_batch.size(0)

        val_loss /= len(val_loader.dataset)

        # Log metrics to wandb
        wandb.log({'train_loss': epoch_loss, 'val_loss': val_loss}, step=epoch)

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = os.path.join(save_dir, 'best_model_checkpoint.pth')
            save_model(model, best_model_path)

        pbar.set_postfix({'Loss': f'{epoch_loss:.4f}', 'Val Loss': f'{val_loss:.4f}'})
        pbar.update(1)

    # Save the last model
    last_model_path = os.path.join(save_dir, 'last_model.pth')
    save_model(model, last_model_path)

    pbar.close()
    print(f'Finished Training. Best validation loss: {best_val_loss:.4f}')
    
    return model

def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model

## Wandb Single

In [10]:
# Initialize wandb
wandb.init(project='RT5307', config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jojaebeom (jaebeom). Use `wandb login --relogin` to force relogin


In [11]:
# 모델 학습
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=500, save_dir=config.SAVE_DIR)

Training model:   0%|          | 0/500 [00:00<?, ?epoch/s]/home/awear-omen/miniforge3/envs/RT5307/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025824022/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
Training model:  29%|██▉       | 146/500 [2:50:08<6:46:17, 68.86s/epoch, Loss=0.0771, Val Loss=0.0905]

RuntimeError: cuDNN error: CUDNN_STATUS_MAPPING_ERROR

# Eval

In [26]:
import torch
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns                           
import numpy as np

In [27]:
def predict(model, data_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    all_probabilities = []
    running_loss = 0.0
    
    with torch.no_grad():
        for X_batch, Y_batch in data_loader:
            X_batch = X_batch.to(device)
            Y_batch = Y_batch.to(device)

            with autocast():
                outputs = model(X_batch)
                if isinstance(outputs, list):  # Deep Supervision
                    outputs = outputs[-1]  # 마지막 출력을 사용
                loss = criterion(outputs, Y_batch)
                running_loss += loss.item() * X_batch.size(0)

                probs = torch.softmax(outputs, dim=2)  # 각 클래스에 대한 확률 계산
                preds = torch.argmax(probs, dim=2)  # 예측값 (원핫 인코딩된 벡터에서 클래스 인덱스로 변환)
                labels = torch.argmax(Y_batch, dim=2)  # 참값 (원핫 인코딩된 벡터에서 클래스 인덱스로 변환)

                all_preds.append(preds.cpu().numpy())
                all_labels.append(labels.cpu().numpy())
                all_probabilities.append(probs.cpu().numpy())
    
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    all_probabilities = np.concatenate(all_probabilities, axis=0)
    avg_loss = running_loss / len(data_loader.dataset)
    
    return all_preds, all_labels, all_probabilities, avg_loss

In [28]:
def plot_confusion_matrix(true_labels, pred_labels, class_names, save_dir):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig(os.path.join(save_dir, 'confusion_matrix.png'))
    plt.close()

In [29]:
def calculate_accuracy(true_labels, pred_labels):
    accuracy = accuracy_score(true_labels, pred_labels)
    return accuracy

In [30]:
def plot_probabilities(true_labels, probabilities, class_names, save_dir, idx):
    num_classes = len(class_names)
    time_steps = probabilities.shape[1]

    fig, axes = plt.subplots(num_classes, 1, figsize=(10, num_classes * 2), sharex=True)

    if num_classes == 1:
        axes = [axes]

    true_labels_one_hot = np.zeros((true_labels.shape[0], true_labels.shape[1], num_classes))
    for i in range(true_labels.shape[0]):
        for t in range(true_labels.shape[1]):
            true_labels_one_hot[i, t, true_labels[i, t]] = 1

    colors = list(mcolors.TABLEAU_COLORS.values())

    for i, class_name in enumerate(class_names):
        color_pred = colors[i % len(colors)]
        color_true = colors[(i + len(colors) // 2) % len(colors)]
        for j in range(true_labels.shape[0]):
            axes[i].plot(range(time_steps), probabilities[j, :, i], label=f'Predicted', alpha=0.6, color=color_pred)
            axes[i].fill_between(range(time_steps), 0, probabilities[j, :, i], alpha=0.2, color=color_pred)
            axes[i].plot(range(time_steps), true_labels_one_hot[j, :, i], linestyle='dashed', label=f'True', alpha=0.6, color=color_true)
            axes[i].fill_between(range(time_steps), 0, true_labels_one_hot[j, :, i], alpha=0.2, color=color_true)
        axes[i].set_ylabel('Probability', fontsize=14)
        axes[i].set_ylim(0, 1)
        axes[i].set_title(f'{class_name}', fontsize=18)
        axes[i].legend(fontsize=14)

    axes[-1].set_xlabel('Time Steps', fontsize=14)

    fig.suptitle(f'{idx}th Result', fontsize=24, y=0.99, x=0.85)
    plt.tight_layout(rect=[0, 0, 1, 1.02])  # tight_layout 제거

    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    save_path = os.path.join(save_dir, f'test_{idx}_probabilities.png')
    plt.savefig(save_path, dpi=300)  # bbox_inches='tight' 옵션 추가
    plt.close()


In [31]:
def plot_probabilities_for_all_trials(probabilities, true_labels, class_names, save_dir):
    total_plots = probabilities.shape[0]
    max_workers = 8 
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        with tqdm(total=total_plots, desc="Plotting probabilities", unit="plot") as progress_bar:
            futures = []
            for num in range(total_plots):
                futures.append(executor.submit(plot_probabilities, true_labels[num:num+1], probabilities[num:num+1], class_names, save_dir, num))
            
            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    print(f"Error occurred: {str(e)}")
                progress_bar.update(1)

In [32]:
class_names = ['ramp ascent', 'ramp descent', 'stair ascent', 'stair descent', 'walk']

In [33]:
model = MultiResUNet.UNet(length=length, model_depth=config.model_depth, num_channel=num_channel, model_width=config.model_width, kernel_size=config.kernel_size, problem_type=config.problem_type, output_channels=output_channels, ds=config.ds, ae=config.ae, feature_number=config.feature_number, is_transconv=config.is_transconv)

criterion = torch.nn.BCEWithLogitsLoss() 

loaded_model = load_model(model, os.path.join(config.SAVE_DIR, 'best_model_checkpoint.pth'))

In [34]:
data_loader = val_loader

pred_labels, true_labels, probabilities, avg_loss = predict(model, data_loader, criterion)

In [35]:
accuracy = calculate_accuracy(true_labels.flatten(), pred_labels.flatten())
print(f"Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

Avg Loss: 0.6218, Accuracy: 0.9763


In [36]:
plot_confusion_matrix(true_labels.flatten(), pred_labels.flatten(), class_names, config.SAVE_DIR)    

In [37]:
plot_probabilities_for_all_trials(probabilities, true_labels, class_names, config.SAVE_DIR)

Plotting probabilities:   1%|          | 4/598 [00:02<04:30,  2.20plot/s]/tmp/ipykernel_7211/3446176364.py:33: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations.
  plt.tight_layout(rect=[0, 0, 1, 1.02])  # tight_layout 제거
Plotting probabilities: 100%|██████████| 598/598 [04:04<00:00,  2.45plot/s]
